# CGM 데이터 분석

### 작성날짜 : 23.10.11

#### 2개의 csv파일 데이터 읽기

In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.family'] = 'Malgun Gothic'

# train_xlsx = pd.read_excel("DA0927_STP1.xlsx")
# train_xlsx.to_csv("cgm_data/in.csv")

cgm_data1 = pd.read_csv('cgm_data/DA0927_STP1.csv',index_col=[0])
cgm_data2 =pd.read_csv('cgm_data/DA0927_STP2.csv',index_col=[0])

C:\Users\ojski\anaconda3\envs\HnsEnv\lib\site-packages\IPython\core\interactiveshell.py:3135: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


#### csv데이터 병합

In [2]:
cgm_datas = pd.DataFrame()
print(len(cgm_data1), len(cgm_data2))

cgm_datas = pd.concat([cgm_datas, cgm_data1])
cgm_datas = pd.concat([cgm_datas, cgm_data2])

print(len(cgm_datas))

59261 42309
101570


#### 혈당량 G(mg/dL) 가 0이하 (0, -1)이거나  Meas가 FALSE값인 경우 데이터 표본에서 제외

In [3]:
cgm_datas.drop(cgm_datas[(cgm_datas['G(mg/dL)'] <= 0) | (cgm_datas['Meas?'] == False)].index, inplace=True)
print(len(cgm_datas))

79006


#### 표 만들기 위한 함수

In [4]:
def make_excel(datas, var_group, var_data):
    count = 0
    for var in var_group:
        var_data[count][0] = datas[datas[var] > 0][var].mean() # [var] [datas[var] > 0][var]
        var_data[count][1] = datas[datas[var] > 0][var].var()
        var_data[count][2] = datas[datas[var] > 0][var].std()
        var_data[count][3] = datas[datas[var] > 0][var].min()
        var_data[count][4] = datas[datas[var] > 0][var].max()
        count += 1
    
    return var_data

def make_newData(cgm_datas):
    cgm_datas.drop(cgm_datas[(cgm_datas['mPD1_dS'] == 0) & (cgm_datas['mPD2_dS'] == 0) & (cgm_datas['mPD3_dS'] == 0) &
                            (cgm_datas['T-rPD_L1dS'] == 0) & (cgm_datas['T-rPD_L2dS'] == 0) & (cgm_datas['T-rPD_L3dS'] == 0) &
                            (cgm_datas['R-rPD_L1dS'] == 0) & (cgm_datas['R-rPD_L2dS'] == 0) & (cgm_datas['R-rPD_L3dS'] == 0) &
                            (cgm_datas['LD123_Bias_Av'] == 0)].index, inplace=True)
    return cgm_datas

#### 환경변수 그룹 12개 + 관심변수 6개 변수에 대한 평균, 분산, 최대최소 값 도출

In [5]:
variable_group1 = ['LD1 Temp', 'LD2 Temp', 'LD3 Temp', 'Rx1 Temp', 'Rx2 Temp', 'S1 T(C)', 'S2 T(C)', 'FR Mon'] 
variable_group2 = ['LD123_Bias_Av', 'mPD1_dS', 'mPD2_dS', 'mPD3_dS']
variable_group3 = ['T-rPD_L1dS', 'T-rPD_L2dS', 'T-rPD_L3dS', 'R-rPD_L1dS', 'R-rPD_L2dS', 'R-rPD_L3dS']

number_data = ['mean', 'var', 'std', 'min', 'max']
# 'G(mg/dL)'

# statistic1 = pd.DataFrame(columns=number_data, index=variable_group1)
# statistic2 = pd.DataFrame(columns=number_data, index=variable_group2)

var_data1 = [[0 for j in range(5)] for i in range(len(variable_group1))]
var_data2 = [[0 for j in range(5)] for i in range(len(variable_group2))]
var_data3 = [[0 for j in range(5)] for i in range(len(variable_group3))]

var_data1 = make_excel(cgm_datas, variable_group1, var_data1)
statistic1 = pd.DataFrame(var_data1, columns=number_data, index=variable_group1)

var_data2 = make_excel(cgm_datas, variable_group2, var_data2)
statistic2 = pd.DataFrame(var_data2, columns=number_data, index=variable_group2)

var_data3 = make_excel(cgm_datas, variable_group3, var_data3)
statistic3 = pd.DataFrame(var_data3, columns=number_data, index=variable_group3)

#### 데이터 csv파일로 저장

In [6]:
cgm_csv = pd.DataFrame()
cgm_csv = pd.concat([cgm_csv, statistic1])
cgm_csv = pd.concat([cgm_csv, statistic2])
cgm_csv = pd.concat([cgm_csv, statistic3])

print(cgm_csv)
# cgm_csv.to_csv('data_analysis.csv',index = True) # csv 파일 저장 코드

                      mean           var         std          min          max
LD1 Temp         26.847898      0.000534    0.023117    26.745000    26.960000
LD2 Temp         26.849628      0.000534    0.023106    26.745000    26.960000
LD3 Temp         26.854912      0.000958    0.030947    26.745000    26.960000
Rx1 Temp         26.268068      0.000511    0.022605    26.208000    26.325000
Rx2 Temp         26.384744      0.000562    0.023709    26.297000    26.449000
S1 T(C)          24.847111      0.843487    0.918415    22.904000    25.957000
S2 T(C)          24.801606      0.756521    0.869782    23.092000    25.847000
FR Mon         1636.825365     81.100814    9.005599  1603.074707  1651.522095
LD123_Bias_Av     1.480366     10.278883    3.206070     0.300000    13.200000
mPD1_dS         979.621185      2.677004    1.636155   977.373711   982.404442
mPD2_dS           0.020226      0.000196    0.013995     0.000681     0.090601
mPD3_dS           0.026666      0.000280    0.016745

In [7]:
print(len(cgm_datas))

new_datas = pd.DataFrame()
new_datas = make_newData(cgm_datas)

print(len(new_datas))

79006
45356
